In [40]:
#seteamos el lugar de trabajo
rootdir="/grandata/ca/voice/"
os.chdir(rootdir)
year = 2012; 

def get_input_file(month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=2012,m=month,d=day)

#levanto la fila de antennas que vamos a droppear
dropped_file = "/home/juan/mobility-study/argentina-scripts/data/celdas_dropped.csv"
dropped_antennas = pd.read_csv(dropped_file)
dropped_antennas = dropped_antennas.values.flatten()
    
#ver el tiempo que tarda
start_time = time.time()

#el chunk basicamente va leyendo el file de a 'chunksize' cantidad de filas
#subgroup = pd.DataFrame()
year = 2012

for month in [1,2,3]:
    days = range(1,32)

    #check february days length
    if month ==2:
        days = range(1,30)

        
    #primero leemos todos los files que vamos a necesitar en ese mes y lo pasamos a formato sframe para cargar mas rapido
    print("Transform all raw datasets to Sframe dirs, time elapsed is {t} ".format(t=(time.time()-start_time)))
    for day in days:
        
        sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)
        
        if not(os.path.exists(sframe_dir)):
            print(sframe_dir)
            local_time = time.localtime()


Transform all raw datasets to Sframe dirs, time elapsed is 0.000338077545166 
Transform all raw datasets to Sframe dirs, time elapsed is 0.0647978782654 
Transform all raw datasets to Sframe dirs, time elapsed is 0.0651140213013 
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/03
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/04
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/05
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/06
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/07
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/08
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/09
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/10
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/11
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/12
/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/03/13
/home/juan/mobility-study/argenti

# SumLinks para argentina
La idea es usar el el raw CDRs dataset para crear el equivalente al sumlinks de mexico que cuenta agrega pares de usuarios y resume informacion de comunicacion entre los 2 usuarios, asi como la distincion de la direccion de uso.

El output tendria que tener el siguiente header:
USER|OTHER_USER|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend


## definiciones utilizadas

* la fecha viene dado en YYYMMDD y el TIME es HHMMSS en formato 24 horas
* *Time* columns are measured in seconds.
* Daylight corresponds to the [8 20) timeframe (using Arg's timezone)
* el analisis es solo para CDRs con usuarios de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming ,1, si entra y viceversa si es outgoing,0).

### el dataset crudo vendria con este header
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF

Basicamente los primeros 2 son los hashes de los users que estuvieron en el call, con el primero el user de la telco.

Direction es quien llama a quien (outgoing es 1-->2, viceversa incoming). Fecha el dia,mes,anyo del llamado, time en formato 24hs incluyendo segundos. Duration medido en segundos, Client_cell_id el hash de la antenna y todo el resto no nos interesa.

### la idea es llegar a algo similar a esto
pero sin la info de sms

In [11]:
!zcat /grandata/voice/sum_links/sum_links_201401_201403.txt.gz | head

LineKeyOrigin|LineKeyTarget|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend|SmsWeekDaylight|SmsWeekNight|SmsWeekend
BA3439E244F09BFA85F03750D84DB1B7|5AA26ADB35AF1A26A635B7747320732F|1|0|0|44|0|0|0|0|0
BA3439E244F09BFA85F03750D84DB1B7|503D0DC95765D42BE5CFB3C50075214A|1|1|0|78|313|0|0|0|0
BA3439E244F09BFA85F03750D84DB1B7|EA6C583D0F31F6D2A1E23B55334DF494|0|0|2|0|0|98|0|0|0
73F169CBB3B6E443B896017C51B1C600|9D0AFC1877F147C63E235CF25B42CC0B|1|4|4|6|85|55|0|0|0
7A0B5B991DD1B3C6E1498C07EF02FD31|9D0AFC1877F147C63E235CF25B42CC0B|0|2|5|0|146|268|0|0|0
EA23ADE2371B8D5FB02A8DDBAF5A53B2|9D0AFC1877F147C63E235CF25B42CC0B|2|1|0|147|46|0|0|0|0
BC834644645F540C1BD72307A5BD5B96|9D0AFC1877F147C63E235CF25B42CC0B|0|0|1|0|0|9|0|0|0
3C43E7D3B99947C9C0C21BA7D387F26A|9D0AFC1877F147C63E235CF25B42CC0B|1|2|0|7|142|0|0|0|0
97B89BED4B7123F6F6F47B41403C4DF1|9D0AFC1877F147C63E235CF25B42CC0B|0|1|0|0|48|0|0|0|0

gzip: stdout: Broken pipe


## Read Raw files

In [1]:
import pandas as pd; import numpy as np; import os;import random;
import graphlab as gl
pd.set_option('display.max_rows', 200)
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
import datetime
np.random.seed(2016)
import subprocess

#seteamos el lugar de trabajo
rootdir="/grandata/ca/voice/"
os.chdir(rootdir)
year = 2012; 

def get_input_file(month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=2012,m=month,d=day)
    
def get_output_file(month,group = -1,sample=False):
    output = "/home/juan/mobility-study/argentina-scripts/output/2012/{0}/sum_links_1".format(month,group)
    if sample == True:
        output = output + "_sample"
    if group != -1:
        output = output + "_group{g}".format(g=group)
    return output



In [20]:
month = 2
#check february days length
if month ==2:
    days = range(1,30)

days = range(1,8)


## First test run

In [21]:
year = 2012
day =1
sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)        
daily_table = gl.load_sframe(sframe_dir)

In [8]:
daily_table.head()

USER,OTHER_USER,DIRECTION,DATE,TIME,CLIENT_CELL_ID
435370362B6A423565774739366A5241416731784B413D3D ...,4C576E38674A724B5475473066396D4D30344C4E6B773D3D ...,O,20120201,093603,CTI 1
4147705945683067353842576E4E302F5472524D67673D3D ...,4C625051683067722B316F46524B6F656B4A6F412F773D3D ...,O,20120201,232018,CF172B
45304C3151446A367376356F424237747168345961773D3D ...,463478573034794730313243555249563234462B77513D3D ...,O,20120201,175446,C1015A
45304C3151446A367376356F424237747168345961773D3D ...,463478573034794730313243555249563234462B77513D3D ...,O,20120201,173419,C1441A
4A6C66694B6D525A6D6C7056546F5A553766346972773D3D ...,454B4C562B4245444C614D4B4C6B6D554E4639706B773D3D ...,O,20120201,194651,CTI 1
4A6C66694B6D525A6D6C7056546F5A553766346972773D3D ...,454B4C562B4245444C614D4B4C6B6D554E4639706B773D3D ...,O,20120201,195151,CTI 1
4A6C66694B6D525A6D6C7056546F5A553766346972773D3D ...,454B4C562B4245444C614D4B4C6B6D554E4639706B773D3D ...,O,20120201,194722,CTI 1
6371745333477372756E52362F5945747A676E7461773D3D ...,79746B2F6739596D79595A65546655553635374B67773D3D ...,O,20120201,221339,C2031G
6371745333477372756E52362F5945747A676E7461773D3D ...,79746B2F6739596D79595A65546655553635374B67773D3D ...,O,20120201,204413,CF270Z
6371745333477372756E52362F5945747A676E7461773D3D ...,79746B2F6739596D79595A65546655553635374B67773D3D ...,O,20120201,204107,C1494G


In [22]:
%%time

## script para extraer todos los atributos del simple_format
#ver el tiempo que tarda
start_time = time.time()

       
#output_file = get_output_file(month,group)
#agrego este control para cuando tuve que cortar antes el algo por alguna razon
#if (os.path.exists(output_file)):
#    print('skipping group {gr} since output-dir exists: {di}'.format(gr = group, di =output_file))

    
#group = 0
#passes = 20
#local_
# print("working group number {it} of {pas}, time elapsed is {t}, localtime is {ho}:{mi} \n".\
#    format(it=group,pas=passes, t=(time.time()-start_time),ho=local_time.tm_hour,mi=local_time.tm_min))
#itero sobre los meses
table =  gl.SFrame()
for day in days:

    sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)        
    daily_table = gl.load_sframe(sframe_dir)
    
    #a cada tabla diaria filtro por todos los USERs por el hash del string (el hash es un int) y despue filtro modulo passes
    #y trabajo sobre la tabla subgroup nada mas que ahora tiene menos usuarios
    #table = table.append(daily_table[daily_table['USER'].apply(lambda x: hash(x) % passes == group )])
    table = table.append(daily_table.head(10**6))
    
#entonces la idea es que yo ahora solo voy a trabajar, dentro de esta tabla filtrada y para todos los dias del mes juntos


CPU times: user 21.6 s, sys: 628 ms, total: 22.2 s
Wall time: 22.1 s


In [23]:
table2 = table.copy()

In [24]:
%%time
#paso las fechas a un timestamp
table['TIMESTAMP'] = table['DATE','TIME'].apply(lambda x: x['DATE'][0:4]+'-'+x['DATE'][4:6]+'-'+x['DATE'][6:8]
                                 +'-'+x['TIME'][0:2]+'-'+x['TIME'][2:4]+'-'+x['TIME'][4:6])

table['TIMESTAMP'] = table['TIMESTAMP'].str_to_datetime(str_format='%Y-%m-%d-%H:%M:%S')

## leo si las llamadas fueron hechas durante el findesemana
table['DURING_WEEKEND'] = table['TIMESTAMP'].apply(lambda x: x.weekday()==5 or x.weekday()==6)

# asigno el bool si la llamada fue en horario 'laboral'
#notar que la hora < 19 significa que 18hs y 59 min da True
table['DURING_DAYLIGHT']= table['TIMESTAMP'].apply(lambda x: x.hour>=8 and x.hour<20 )

CPU times: user 55.7 s, sys: 2.68 s, total: 58.4 s
Wall time: 8.8 s


In [25]:
table['DURING_WEEKEND'].show()

In [26]:
table['DURING_DAYLIGHT'].show()

CallsWeekDaylight | CallsWeekNight | CallsWeekend | TimeWeekDaylight | TimeWeekNight | TimeWeekend


In [16]:
table_weeknight = table[(table['DURING_DAYLIGHT']==False) &  (table['DURING_WEEKEND']==False)]

In [17]:
table_weeknight.shape, table.shape

((1385639, 11), (7000000, 11))

In [27]:
%%time
table_weekend = table[table['DURING_WEEKEND']==True]

table_weekend_call = table_weekend.groupby(['USER','OTHER_USER'],
                 {'CallsWeekEnd':gl.aggregate.COUNT('TIMESTAMP')})

table_weeknight = table[(table['DURING_DAYLIGHT']==False) &  (table['DURING_WEEKEND']==False)]

table_weeknight_call = table_weeknight.groupby(['USER','OTHER_USER'],
                 {'CallsWeekNight':gl.aggregate.COUNT('TIMESTAMP')})

table_weekday = table[(table['DURING_DAYLIGHT']==True) &  (table['DURING_WEEKEND']==False)]

table_weekday_call = table_weekday.groupby(['USER','OTHER_USER'],
                 {'CallsWeekDay':gl.aggregate.COUNT('TIMESTAMP')})

table_weekend_time = table_weekend.groupby(['USER','OTHER_USER'],
                 {'TimeWeekEnd':gl.aggregate.SUM('DURATION')})


table_weeknight_time = table_weeknight.groupby(['USER','OTHER_USER'],
                 {'TimeWeekNight':gl.aggregate.SUM('DURATION')})


table_weekday_time = table_weekday.groupby(['USER','OTHER_USER'],
                 {'TimeWeekDay':gl.aggregate.SUM('DURATION')})

del table_weekend, table_weeknight, table_weekday

CPU times: user 2min 28s, sys: 15 s, total: 2min 43s
Wall time: 49.1 s


In [28]:
# ahora mergeamos todas las tablas anteriores con un outer join en c/ casi Y FILLNA
#recordar que para graphlab el fillna es por columna!

In [29]:
%%time
table = table_weekend_call.join(table_weeknight_call,on = ['USER','OTHER_USER'], how = 'outer')
del table_weekend_call, table_weeknight_call
table =  table.join(table_weekday_call,on = ['USER','OTHER_USER'], how = 'outer')
del table_weekday_call

table =  table.join(table_weekend_time,on = ['USER','OTHER_USER'], how = 'outer')
del table_weekend_time

table =  table.join(table_weekday_time,on = ['USER','OTHER_USER'], how = 'outer')
del table_weekday_time

table =  table.join(table_weeknight_time,on = ['USER','OTHER_USER'], how = 'outer')
del table_weeknight_time

for col in table.column_names():
    if 'USER' in col:
        continue
    table = table.fillna(col,0)

CPU times: user 2min 19s, sys: 9.38 s, total: 2min 28s
Wall time: 43.9 s


In [37]:
table.sample(1e-5)

OTHER_USER,USER,CallsWeekEnd,CallsWeekNight,CallsWeekDay,TimeWeekEnd
586E776E42424B6D6C5A61327678782F7353574467413D3D ...,49344F567A377973674D772F6872354E6A4F44524C413D3D ...,0,0,2,0
343134702F3879322F4E313133784247644E4B4130413D3D ...,7945302B7956577847504D672B5A73346659316239773D3D ...,0,0,1,0
6F465559654D4D434B42474B497A39434B59506661513D3D ...,6F465559654D4D434B424879504C33374437305754513D3D ...,0,0,2,0
694279704E436255783563376D593369476E7A6F67673D3D ...,556449596230397444506B2B58583731324B32512B413D3D ...,0,0,1,0
4465314D6E7566716E476468507345537A48577247513D3D ...,61384F504732415A3852666968796D746B7A686F2F413D3D ...,0,0,1,0
392F45313274627036774B57492F397179736A366A673D3D ...,70437053556F4B537A504234773473346353385478413D3D ...,0,0,1,0
3968426F79792B5A32314C6A736F46505934786F44413D3D ...,5050682F397A7853486362306670414852762F7148673D3D ...,0,0,1,0
744C3768582F492F43596B79344572424C4357704E513D3D ...,7871434E6F6F764B7A394F426E4A47704E71634C62773D3D ...,0,0,1,0
53795670676F2B6842442B2B31445142346B363633773D3D ...,74517A45674E52652B50376B55357A302F2F4D7251413D3D ...,0,0,1,0
384170506C68676C376F326972693764656E465534513D3D ...,314A78736D306952342B57784F6153325433364E52513D3D ...,0,0,2,0


In [31]:
table['CallsWeekDay'].show()

In [32]:
table['CallsWeekNight'].show()

In [33]:
table['CallsWeekEnd'].show()

In [34]:
table['TimeWeekDay'].show()

In [35]:
table['TimeWeekNight'].show()

In [36]:
table['TimeWeekEnd'].show()

## consitency check from table2
* check we're not losing unique num of user/other_users
* check random user samples

In [40]:
%%time
total_users_in = len(table2.groupby('USER',
                 {'count':gl.aggregate.COUNT('DURATION')}))
total_users_out = len(table.groupby('USER',
                 {'count':gl.aggregate.COUNT('CallsWeekNight')}))
print(total_users_in,total_users_out)

(1880230, 1880230)
CPU times: user 1min 13s, sys: 3.01 s, total: 1min 16s
Wall time: 19.5 s


In [78]:
%%time
total_ousers_in = len(table2.groupby('OTHER_USER',
                 {'count':gl.aggregate.COUNT('DURATION')}))
total_ousers_out = len(table.groupby('OTHER_USER',
                 {'count':gl.aggregate.COUNT('CallsWeekNight')}))
print(total_ousers_in,total_ousers_out)

(2098263, 2098263)
CPU times: user 1min 23s, sys: 3.41 s, total: 1min 26s
Wall time: 22 s


In [88]:
table2['USER'].sample(0.01).head(10)[0]

'5030367459586451306D655135545635714337704C513D3D'

In [89]:
usr = "5030367459586451306D655135545635714337704C513D3D"

before = table2[table2['USER']==usr][['USER','OTHER_USER','DATE','TIME','DURATION']]

In [90]:
after = table[table['USER']==usr]

In [91]:
set(before['OTHER_USER'].unique()) == set(after['OTHER_USER'].unique())

True

In [92]:
set(before['USER'].unique()) == set(after['USER'].unique())

True

In [93]:
# wednesday 01/02 thu 02/02 fri 03/02 sat 04/02 sun 05/02 mon 06/02 tue 07/02

In [94]:
before

USER,OTHER_USER,DATE,TIME,DURATION
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,223333,476
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,225210,80
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,224653,132
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,213355,488
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,224245,144
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,222837,44
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,162914,132
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,153749,3011
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120201,153605,1
5030367459586451306D655135545635714337704C513D3D ...,474A6D773639717256495A66582B637347704C5176773D3D ...,20120203,215933,1


In [95]:
# wednesday 01/02 thu 02/02 fri 03/02 sat 04/02 sun 05/02 mon 06/02 tue 07/02

In [96]:
after

Columns:
	OTHER_USER	str
	USER	str
	CallsWeekEnd	int
	CallsWeekNight	int
	CallsWeekDay	int
	TimeWeekEnd	int
	TimeWeekDay	int
	TimeWeekNight	int

Rows: Unknown

Data:
+-------------------------------+-------------------------------+--------------+
|           OTHER_USER          |              USER             | CallsWeekEnd |
+-------------------------------+-------------------------------+--------------+
| 474A6D773639717256495A6658... | 5030367459586451306D655135... |      1       |
+-------------------------------+-------------------------------+--------------+
+----------------+--------------+-------------+-------------+---------------+
| CallsWeekNight | CallsWeekDay | TimeWeekEnd | TimeWeekDay | TimeWeekNight |
+----------------+--------------+-------------+-------------+---------------+
|       8        |      3       |      2      |     3144    |      1367     |
+----------------+--------------+-------------+-------------+---------------+
[? rows x 8 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.

In [5]:
month =1
group =10
output_file = get_output_file(month,group)
print('output_file is located at %s' % output_file)


output_file is located at /home/juan/mobility-study/argentina-scripts/output/2012/1/sum_links_1_group10


In [2]:
local_time = time.localtime()       
print('finished data processing from group {g} of {p}, time elapsed is {t}\n Start saving output files , localtime is {ho}:{mi} \n'.format(g = group, p = passes, t=(time.time()-start_time)),
                  ho=local_time.tm_hour,mi=local_time.tm_min)
#salvo c/file por separado y despues los appendeo todos en 1.
output_file = get_output_file(month,group)
print('output_file is located at %s' % output_file)

table.save(output_file)

#comprimo a mano el archivo pues SFrame no comprime en gzip automaticamente los csvs
#bashCommand = "gzip {0}".format(output_file)
#process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
#output = process.communicate()[0]    
#print('Output of bash gzip command is {0}'.format(output))

local_time = time.localtime()
print("Finished group {gr} of {p}, time elapsed is {t}\, localtime is {ho}:{mi}\n ".\
              format(gr=group,p=passes,t=(time.time()-start_time),ho=local_time.tm_hour,mi=local_time.tm_min))

then = start_time
seconds = time.time() - then
print("total running time of script is %d " % seconds)

Transform all raw datasets to Sframe dirs, time elapsed is 0.000231981277466 


In [107]:
!ls /home/juan/mobility-study/argentina-scripts/output/2012/1/

sum_links_1			user_antenna_mapping_group18_1
sum_links_1_sample		user_antenna_mapping_group19_1
user_antenna_mapping_group0_1	user_antenna_mapping_group1_1
user_antenna_mapping_group10_1	user_antenna_mapping_group2_1
user_antenna_mapping_group11_1	user_antenna_mapping_group3_1
user_antenna_mapping_group12_1	user_antenna_mapping_group4_1
user_antenna_mapping_group13_1	user_antenna_mapping_group5_1
user_antenna_mapping_group14_1	user_antenna_mapping_group6_1
user_antenna_mapping_group15_1	user_antenna_mapping_group7_1
user_antenna_mapping_group16_1	user_antenna_mapping_group8_1
user_antenna_mapping_group17_1	user_antenna_mapping_group9_1


In [13]:
def get_input_file(month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=2012,m=month,d=day)

antenna_file = "/home/juan/mobility-study/argentina-scripts/data/celdas_dropped.csv"

dropped_antennas = pd.read_csv(antenna_file)

dropped_antennas = dropped_antennas.values.flatten()

In [16]:
year = 2012
month =2
day = 15

In [18]:
start_time = time.time()
sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)

print("Reading disk file for day {d}-{ms}, time elapsed is {t} ".format(d=day,ms=month,t=(time.time()-start_time)))

input_file= get_input_file(month,day)

daily_table = gl.SFrame.read_csv( input_file, delimiter='|', 
        header=True, skip_initial_space=True, 
        column_type_hints = [str,str, str,str, str, str,str, int], 
        na_values=['NaN'],
        usecols = ['ORIGIN_NUMBER_ENC_B64',"TARGET_NUMBER_ENC_B64",'DIRECTION',
                            'FECHA','TIME','CLIENT_CELL_ID','OTHER_OPERATOR','DURATION'],  
        error_bad_lines=False,
        verbose = False
                  )

daily_table.rename({'ORIGIN_NUMBER_ENC_B64':'USER','FECHA':'DATE',
'TARGET_NUMBER_ENC_B64':'OTHER_USER'})

Reading disk file for day 15-2, time elapsed is 0.000160932540894 


/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-06-05 23:38:18,956 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1465169896.log


USER,OTHER_USER,DIRECTION,DATE,TIME,CLIENT_CELL_ID
334446724A587750697A7647713565795639704C31513D3D ...,54334B6D65787472554B586758676C593745536836773D3D ...,O,20120215,125535,NQ065C
334446724A587750697A7647713565795639704C31513D3D ...,54334B6D65787472554B586758676C593745536836773D3D ...,O,20120215,125444,NQ065C
4A6478585253587A594F6D437175312F734C6E6678773D3D ...,6A7672526342564E744F46375054536973794C716C513D3D ...,O,20120215,124358,CO647B
39507373392F51742B6B564B584E5A50586253674C413D3D ...,4171486C5876507A6E72354C7430524D5159384E79513D3D ...,O,20120215,120646,CR103Z
37655235494335702B3865385656426A752B566448413D3D ...,66634967306D705A423366376B6C4436667A4F4433413D3D ...,O,20120215,134927,SF480B
37655235494335702B3865385656426A752B566448413D3D ...,66634967306D705A423366376B6C4436667A4F4433413D3D ...,O,20120215,135427,SF480B
784C4E7A6C35524131327950766364566537456759513D3D ...,734E322F716C6A5A4D4C584B59576F584A6377686A773D3D ...,O,20120215,135756,CF124G
6950545775504F2B737658653374514E696C587077673D3D ...,4B4858345A4B32354175756D32427A4F7A5A357438413D3D ...,O,20120215,124239,TU091E
655859782B63376949674B344E353165513754696B513D3D ...,655859782B63376949674A794F344235754F462F54673D3D ...,O,20120215,124228,CF621Z
3467466E4C6C383350456630726C32396C6F6D555A413D3D ...,533730534A7A317A676F2B79673872306356615875513D3D ...,O,20120215,125020,C2324A


In [37]:
%%time
#checking which antennas ARENT in the dropped list
ind = daily_table['CLIENT_CELL_ID'].apply(lambda x: x in dropped_antennas)
ind = (ind==0)
daily_table = daily_table[ind]

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 30 ms


In [39]:

daily_table

USER,OTHER_USER,DIRECTION,DATE,TIME,CLIENT_CELL_ID
65716F4258355674736B4575302F4233485851716B513D3D ...,3154314D77494B7556665576526D4D635963723341673D3D ...,O,20120215,152945,C1893Z
724C33306E58763949793974436A59796E484F6E78673D3D ...,3630704C416A3050506A7436634D7A743341673076773D3D ...,O,20120215,183942,SF984G
5170482F3871786639425966735862307344646478413D3D ...,454C384E3275706549696141364A676B7236304144673D3D ...,O,20120215,222241,RJ209Z
46556C7A737A756349667263494861697130506A33513D3D ...,2F642B37344E2B376B7A6E325835797A47466A4F42773D3D ...,O,20120215,100423,SF430B
53525932544D51674779393631515A786D4C4C3465773D3D ...,37782F5A795565462B6C624450397679786B726571513D3D ...,O,20120215,165901,C1784G
67634776565932744F736D75735A7168305850316D513D3D ...,474E52726D6F2F634B2B73694F4531616970325338673D3D ...,O,20120215,114702,SC046A
67634776565932744F736D75735A7168305850316D513D3D ...,474E52726D6F2F634B2B73694F4531616970325338673D3D ...,O,20120215,114728,SC046D
7A313268506162554364416D3678745A4244353968513D3D ...,4B574B4375736D73642F5658674D68477058636A51413D3D ...,O,20120215,111524,C2173G
7A313268506162554364416D3678745A4244353968513D3D ...,4B574B4375736D73642F5658674D68477058636A51413D3D ...,O,20120215,111453,C2173G
7563554A79614D4C53634A4E7737677A5648756277673D3D ...,7563554A79614D4C53634936716E434D7565433359413D3D ...,O,20120215,212659,C2311A


In [28]:
out = daily_table.head(10)['CLIENT_CELL_ID'].apply(lambda x: x in dropped_antennas )
out

dtype: int
Rows: 10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [35]:
out.astype(np.bool) ==0

dtype: int
Rows: 10
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [29]:
out.dtype

<bound method SArray.dtype of dtype: int
Rows: 10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]>

In [26]:
out[0] in dropped_antennas

False